In [1]:
import os
import re
import sqlite3
import requests

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [2]:
columns = [
  'Player',
  'Tm',
  'Age',
  'Pos',
  'G',
  'GS',
  'QBrec',
  'Cmp',
  'Att',
  'Cmp%',
  'Yds',
  'TD',
  'TD%',
  'Int',
  'Int%',
  '1D',
  'Lng',
  'Y/A',
  'AY/A',
  'Y/C',
  'Y/G',
  'Rate',
  'QBR',
  'Sk',
  'Yds2',
  'Sk%',
  'NY/A',
  'ANY/A',
  '4QC',
  'GWD',
]

In [3]:
def parse(html, season):
    data = []

    bs = BeautifulSoup(html)
    for row in bs.select('#div_passing tbody tr'):
        tds = row.select('td')

        if len(tds) != len(columns):
            continue

        data.append(
            [td.text for td in tds] + [season]
        )

    return data

In [4]:
data = []
for season in (str(year) for year in range(2017, 2023)):
    response = requests.get(f'https://www.pro-football-reference.com/years/{season}/passing.htm')
    assert response.status_code == 200
    
    observations = parse(response.text, season)
    data.extend(observations)

In [5]:
df = pd.DataFrame(data, columns=columns + ['season'])
df.head()

,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Rate,QBR,Sk,Yds2,Sk%,NY/A,ANY/A,4QC,GWD,season
0,Tom Brady*+,NWE,40,QB,16,16,13-3-0,385,581,66.3,...,102.8,70.6,35,201,5.7,7.10,7.56,2,2,2017
1,Philip Rivers*,LAC,36,QB,16,16,9-7-0,360,575,62.6,...,96.0,61.1,18,120,3,7.41,7.60,2,2,2017
2,Matthew Stafford,DET,29,QB,16,16,9-7-0,371,565,65.7,...,99.3,65.2,47,287,7.7,6.80,7.01,1,4,2017
3,Drew Brees*,NOR,38,QB,16,16,11-5-0,386,536,72,...,103.9,62.5,20,145,3.6,7.53,7.71,2,2,2017
4,Ben Roethlisberger*,PIT,35,QB,15,15,12-3-0,360,561,64.2,...,93.4,66.6,21,139,3.6,7.07,6.95,3,4,2017


In [6]:
def build_bulk_insert_sql(df, table_name):
    sql = f"""INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES"""

    dtypes = list(df.dtypes.to_dict().items())
    df = df.fillna(np.nan).replace([np.nan], [None])

    for _, row in df.iterrows():
        insertable = []
        for col, dtype in dtypes:
            value = row[col]
            if dtype.name == 'object' and not value is None:
                insertable.append('"' + str(row[col]).strip() + '"')
            elif value is None:
                insertable.append('NULL')
            else:
                insertable.append(str(row[col]))

        insertable_sql = ', '.join(insertable)
        sql += f'\n({insertable_sql}),'

    return sql[:-1]

In [7]:
def clean_name(text):
    text = re.sub(r'[+* ]+$', '', text)
    text = text.strip()
    return text

quarterbacks = df[['Player', 'Cmp', 'Att', 'Yds', 'Int', 'TD', 'season']].copy()
quarterbacks.columns = ['name', 'completions', 'attempts', 'yards', 'interceptions', 'touchdowns', 'season']

quarterbacks['name'] = quarterbacks['name'].map(clean_name)

quarterbacks['completions'] = quarterbacks['completions'].astype(int)
quarterbacks['attempts'] = quarterbacks['attempts'].astype(int)
quarterbacks['yards'] = quarterbacks['yards'].astype(int)
quarterbacks['interceptions'] = quarterbacks['interceptions'].astype(int)
quarterbacks['touchdowns'] = quarterbacks['touchdowns'].astype(int)

quarterbacks.head()

,name,completions,attempts,yards,interceptions,touchdowns,season
0,Tom Brady,385,581,4577,8,32,2017
1,Philip Rivers,360,575,4515,10,28,2017
2,Matthew Stafford,371,565,4446,10,29,2017
3,Drew Brees,386,536,4334,8,23,2017
4,Ben Roethlisberger,360,561,4251,14,28,2017


In [8]:
quarterbacks.dtypes

name             object
completions       int64
attempts          int64
yards             int64
interceptions     int64
touchdowns        int64
season           object
dtype: object

In [9]:
insert_sql = build_bulk_insert_sql(quarterbacks, 'quarterbacks')

In [10]:
create_table_sql = """
  CREATE TABLE quarterbacks(
    name VARCHAR(200) NOT NULL,
    season VARCHAR(50) NOT NULL,
    completions NUMERIC NOT NULL,
    attempts NUMERIC NOT NULL,
    yards NUMERIC NOT NULL,
    interceptions NUMERIC NOT NULL,
    touchdowns NUMERIC NOT NULL,

    PRIMARY KEY (name, season)
  );
"""

In [11]:
if os.path.exists('../../data/nfl/quarterbacks.sqlite3'):
    os.remove('../../data/nfl/quarterbacks.sqlite3')

with sqlite3.connect('../../data/nfl/quarterbacks.sqlite3') as conn:
    cur = conn.cursor()
    cur.execute(create_table_sql)
    cur.execute(insert_sql)

In [12]:
with sqlite3.connect('../../data/nfl/quarterbacks.sqlite3') as conn:
    cur = conn.cursor()
    cur.execute('select * from quarterbacks;')

    records = cur.fetchall()

    for row in records[:5]:
        print(row)

    for row in records[-5:]:
        print(row)

('Tom Brady', '2017', 385, 581, 4577, 8, 32)
('Philip Rivers', '2017', 360, 575, 4515, 10, 28)
('Matthew Stafford', '2017', 371, 565, 4446, 10, 29)
('Drew Brees', '2017', 386, 536, 4334, 8, 23)
('Ben Roethlisberger', '2017', 360, 561, 4251, 14, 28)
('DeeJay Dallas', '2022', 0, 1, 0, 1, 0)
('Chad Henne', '2022', 0, 2, 0, 0, 0)
('Christian Kirk', '2022', 0, 1, 0, 0, 0)
('Cooper Kupp', '2022', 0, 1, 0, 0, 0)
('Tommy Townsend', '2022', 0, 1, 0, 0, 0)
